In [1]:
# Imports
import argparse
import os
import sys
import time
import glob
import re

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from _datetime import datetime
import scipy.stats as stats
import matplotlib.animation as animation
from matplotlib import rcParams
from PIL import Image
from sklearn.utils import resample
from scipy.stats import pearsonr
import scipy.stats as stats
import scipy

In [2]:
# Local imports
sys.path.append("/home/users/benhutch/skill-maps")
import dictionaries as dicts

sys.path.append("/home/users/benhutch/skill-maps/python")
import functions as fnc

In [3]:
# Imports of historical functions
#/home/users/benhutch/skill-maps-differences/functions.py
import os

path = "/home/users/benhutch/skill-maps-differences"
print(os.listdir(path))

# append path and import
sys.path.append(path)
import functions_diff as hist_fnc

# List the functions in the module
print(dir(hist_fnc))

['.git', '.gitignore', 'README.md', 'functions_diff.py', 'dictionaries.py', 'plotting.ipynb', '__pycache__', 'plotting_functions.py', '.ipynb_checkpoints', 'skill_maps_functions.py']
['CDOException', 'CDO_PY_VERSION', 'Cdo', 'CdoTempfileStore', 'Image', 'StringIO', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'animation', 'argparse', 'auto_doc', 'calculate_annual_mean_anomalies_obs', 'calculate_anomalies', 'calculate_correlation_coefficient', 'calculate_correlations', 'calculate_correlations_diff', 'calculate_dcpp_data_ws', 'calculate_historical_data_ws', 'calculate_spatial_correlations', 'calculate_spatial_correlations_bootstrap', 'calculate_spatial_correlations_bootstrap_diff', 'calculate_spatial_correlations_diff', 'calculate_wind_speed', 'ccrs', 'cdo', 'constrain_years_init_uninit', 'constrain_years_processed_hist', 'da', 'dask', 'datetime', 'dd', 'dic', 'extract_ensemble_members', 'extract_historical_data', 'getCdoVersio

In [4]:
# Set up the parameters
# Test run with psl dcppA-hindcast raw data and historical data
variable = "psl"
obs_var_name = "psl"
region = "global"
region_grid = dicts.gridspec_global
forecast_range = "2-9"
season = "DJFM"
model_season = "DJFM"
start_year = "1960"
end_year = "2022"

# Set up the paths
observations_path = dicts.obs
dcpp_models = dicts.models
historical_models = dicts.psl_models_historical
base_dir = dicts.base_dir
base_dir_historical = dicts.base_dir_historical
output_dir = dicts.plots_dir
save_dir = dicts.save_dir

In [5]:
%tb
# Process the observations
obs = fnc.process_observations(variable, region, region_grid, forecast_range, season,
                                observations_path, obs_var_name, plev=None)

Gridspec file: /home/users/benhutch/gridspec/gridspec-global.txt
Variable is not ua or va, creating new file name
File already exists
Loading ERA5 data


No traceback available to show.


In [6]:
# Load the historical data
hist_data = hist_fnc.load_processed_historical_data(base_dir_historical, historical_models,
                                                    variable, region, forecast_range, season)

# Extract the data for the variable from the historical data
hist_data, _ = hist_fnc.extract_historical_data(hist_data, variable)

processing model:  BCC-CSM2-MR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/BCC-CSM2-MR/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  MPI-ESM1-2-HR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/MPI-ESM1-2-HR/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  CanESM5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/CanESM5/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  CMCC-CM2-SR5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/CMCC-CM2-SR5/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  HadGEM3-GC31-MM
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/HadGEM3-GC31-MM/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  MPI-ESM1-2-LR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/MPI-ESM1-2-LR/global/years_2-9/DJFM/outputs/processed/*.nc
processing m

In [7]:
# Load the DCPP data
dcpp_data = fnc.load_data(base_dir, dcpp_models, variable, region, 
                            forecast_range, season, level=None)

# Extract the data for the variable from the DCPP data
dcpp_data, _ = fnc.process_data(dcpp_data, variable)

Skipping file /home/users/benhutch/skill-maps-processed-data/psl/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_psl_global_2-9_DJFM-r7i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/psl/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_psl_global_2-9_DJFM-r6i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/psl/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_psl_global_2-9_DJFM-r9i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/psl/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_psl_global_2-9_DJFM-r8i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/psl/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_psl_global_2-9_DJFM-r10i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/psl/FGOALS-f3-L/global/years_2-9/DJFM/outputs/mergetime/mergetime_FGOALS-f3-L_psl_global_2-9_DJFM-r3i1.nc
Skipping file /ho

In [8]:
# Now we want to make sure that the time period 
# Use constrain years to make sure that the model data is the same
constrained_hist_data = fnc.constrain_years(hist_data, historical_models)

# Constrain the years for the DCPP data
constrained_dcpp_data = fnc.constrain_years(dcpp_data, dcpp_models)

# Align the forecasts and observations
fcst1, fcst2, obs, common_years = fnc.align_forecast1_forecast2_obs(constrained_dcpp_data, dcpp_models, 
                                                                    constrained_hist_data, historical_models, 
                                                                    obs)


There is a gap of more than 1 year in the years for model CanESM5member r3i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r19i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r15i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r1i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r18i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r10i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r13i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r3i1p2f1
adjusted indices: 54
years where gap is greater than 1: [2019 2020 2021 2023 2024]
There is a gap of more than 1 year in the years for model CanESM5member r19i1p2f1
adjusted indices: 54
years where gap is greater than 1: [2019 2020 2021 2023 2024]
There is a gap of more than 1 year in the years for model CanESM5member r15i1p2f1
adjusted indices:

In [9]:
# Now we can test the bootstrapping function
forecast_stats = fnc.forecast_stats(obs, fcst1, fcst2)

bootstrap index 0
bootstrap index 1
bootstrap index 2
bootstrap index 3
bootstrap index 4


In [11]:
print(forecast_stats)

{'corr1': array([[0.63937347, 0.63937323, 0.63937338, ..., 0.63937303, 0.63937319,
        0.63937343],
       [0.64059046, 0.65190366, 0.66301066, ..., 0.60689121, 0.61736793,
        0.62928298],
       [0.79574508, 0.79581387, 0.80457398, ..., 0.78245657, 0.7903488 ,
        0.79615478],
       ...,
       [0.2238035 , 0.22212076, 0.22038002, ..., 0.22843081, 0.22742408,
        0.22579965],
       [0.22389237, 0.22251622, 0.22094549, ..., 0.22551915, 0.22543049,
        0.22465157],
       [0.20180229, 0.20071417, 0.19954833, ..., 0.20410923, 0.20358821,
        0.20276663]]), 'corr1_min': array([[0.26315487, 0.26315433, 0.26315507, ..., 0.26315425, 0.26315488,
        0.26315555],
       [0.31507766, 0.33000611, 0.34713835, ..., 0.26724899, 0.2816387 ,
        0.29808662],
       [0.60577737, 0.61026368, 0.62664765, ..., 0.57544785, 0.59021914,
        0.60281454],
       ...,
       [0.10044043, 0.09907357, 0.09752304, ..., 0.10515131, 0.10337396,
        0.1017293 ],
       [0.1